# AFL Predictions
The file we will run to get each rounds predictions!

In [1]:
#packages
import polars as pl
import pickle
import numpy as np
import pandas as pd
#Get Update function
from ipynb.fs.defs.AFLupdate import updateData, getCurrentRoundNumbers
#Get upcoming fixture
from ipynb.fs.defs.AFLfixture import createFixture

pl.Config.set_fmt_str_lengths(150);
pl.Config.set_tbl_cols(40);
pl.Config.set_tbl_rows(50);


/home/erin/Documents/AFL tipping/afl-env/lib/python3.12/site-packages/rpy2/robjects/lib/dplyr.py:27: UserWarning: This was designed against dplyr versions starting with 1.0 but you have 1.1.4
  warnings.warn(


In [2]:
#Run updates
updateData()

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

No update Required


In [3]:
#get the fixture 
fixture, predictionDF = createFixture()

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

ℹ Returning data for "Round 10, 2025"
✔ Returning data for "Round 10, 2025" s]

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

ℹ Returning data for "Round 9, 2025"
✔ Returning data for "Round 9, 2025" s]

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

ℹ Fetching match ids
✔ Fetching match ids s]

ℹ Finding player stats for 216 matches.
✔ Finding player stats for 216 matches. [46.5s]

ℹ Fetching match ids
✔ Fetching match ids s]

ℹ Finding player stats for 207 matches.
✔ Finding player stats for 207 matches. [29.1s]



In [4]:
#if fixture exists i.e no issues have been introduced, go ahead and use our models to get our predictions
modelWin = pickle.load(open('Data/winner.sav', 'rb'))

predictWinner = modelWin.predict(fixture).tolist()
probsWinner = modelWin.predict_proba(fixture).tolist()

#create the winner portion of the prediction df
predictionDF = (predictionDF
                 #place in results
                 .with_columns(pl.Series(predictWinner).alias('Predictions'),
                               pl.Series(probsWinner).alias('Probs'))
                 #Winner probs
                 .with_columns(((pl.col('Probs').list.get(pl.col('Predictions')))*100).round(2).alias('Win Prob'))
                 .with_columns(pl.when(pl.col('Predictions') == 1).then(pl.col('home.team.name')).otherwise(pl.col('away.team.name')).alias('Predicted Winner'))
                 #Add current round
                 .with_columns(pl.lit(getCurrentRoundNumbers()).alias('Round Number'))
                 #rename
                 .rename({'home.team.name' : 'Home Team',
                          'away.team.name' : 'Away Team'})
                 #keep only wants required
                 .select('Round Number', 'Home Team', 'Away Team', 'Predicted Winner', 'Win Prob')
                )


ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]



In [5]:
predictionDF

Round Number,Home Team,Away Team,Predicted Winner,Win Prob
i32,str,str,str,f64
10,"""Gold Coast Suns""","""Hawthorn""","""Hawthorn""",63.72
10,"""Sydney Swans""","""Carlton""","""Sydney Swans""",57.99
10,"""Collingwood""","""Adelaide Crows""","""Collingwood""",68.12
10,"""Port Adelaide""","""Geelong Cats""","""Port Adelaide""",78.62
10,"""GWS Giants""","""Fremantle""","""GWS Giants""",52.2
10,"""Western Bulldogs""","""Essendon""","""Western Bulldogs""",62.64
10,"""Richmond""","""North Melbourne""","""Richmond""",54.92
10,"""Brisbane Lions""","""Melbourne""","""Brisbane Lions""",63.6
10,"""West Coast Eagles""","""St Kilda""","""St Kilda""",60.14


In [6]:
#Get the margin
#load the model in
model = pickle.load(open('Data/margin.sav', 'rb'))

#And the win probablities to fixture
fixture[['homeProb', 'awayProb']] = probsWinner
#predict the fixture
predictMargin = model.predict(fixture).tolist()

#Add to prediction df
predictionDF = (predictionDF.with_columns(pl.Series(predictMargin).round(0).cast(pl.Int32).alias('Margin')))


In [7]:
predictionDF

Round Number,Home Team,Away Team,Predicted Winner,Win Prob,Margin
i32,str,str,str,f64,i32
10,"""Gold Coast Suns""","""Hawthorn""","""Hawthorn""",63.72,35
10,"""Sydney Swans""","""Carlton""","""Sydney Swans""",57.99,25
10,"""Collingwood""","""Adelaide Crows""","""Collingwood""",68.12,31
10,"""Port Adelaide""","""Geelong Cats""","""Port Adelaide""",78.62,46
10,"""GWS Giants""","""Fremantle""","""GWS Giants""",52.2,35
10,"""Western Bulldogs""","""Essendon""","""Western Bulldogs""",62.64,26
10,"""Richmond""","""North Melbourne""","""Richmond""",54.92,19
10,"""Brisbane Lions""","""Melbourne""","""Brisbane Lions""",63.6,21
10,"""West Coast Eagles""","""St Kilda""","""St Kilda""",60.14,26


In [8]:
#Write to folder
#Individual round tips
predictionDF.write_csv(fr'Tips/2025/Round_{getCurrentRoundNumbers()}.csv')

#Append to all_tips
with open("Tips/2025/All_Tips.csv", mode="a") as f:
   predictionDF.write_csv(f, include_header=False)

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

